In [0]:
%sql
/*
NoteBook que simula el Job JP_PATRIF_GAR_0500_INS_ARCHIVO Que solamente genera la insercion de datos de hacia CIERREN.TTAFOGRAL_ARCHIVO del ambiente de OCI
SUBPROCESO 365
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("PRUEBA")).orderBy("Archivo SQL"))

In [0]:
'''
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual = datetime.now(mexico_tz).strftime("%Y%m%d")
#fecha timeStamp
fecha_actual_sin_formato = datetime.now(mexico_tz).strftime("%d-%m-%Y %H:%M:%S.%f")
'''

In [0]:
statement_001 = query.get_statement(
"PRUEBA.PIVOT.sql",
SR_FOLIO=params.sr_folio,
)

In [0]:
df = db.read_data("default", statement_001)
#if conf.debug:
display(df)

In [0]:
#from pyspark.sql.functions import expr


df_inserta_archivo = df.select(
    #col("FTN_ID_ARCHIVO"),
    col("FTC_NOMBRE"),
    col("FCN_ID_PROCESO"),
    col("FCN_ID_SUBPROCESO"),
    col("FTD_FEH_GEN"),
    col("FTD_FEH_CRE"),
    col("FTC_USU_CRE"),
    col("FTD_FEH_ARCHIVO"),
    col("FCN_ID_ESTATUS_ARCHIVO")
    )



display(df_inserta_archivo)


In [0]:
#Carga los datos del primer dataframe a la tabla CIERREN.TTAFOGRAL_ARCHIVO
db.write_data(df_inserta_archivo, "CIERREN.TTAFOGRAL_ARCHIVO", "default", "append")



In [0]:
df_inserta_archivo_folio = df.select(
    #col("FTN_ID_ARCHIVO"),
    (col("FTN_ID_ARCHIVO").cast('int')-1).alias("FTN_ID_ARCHIVO"),
    #(col("FTN_ID_ARCHIVO").cast("int") + 1).alias("FTN_ID_ARCHIVO"),
    col("FTC_FOLIO")
    )

display(df_inserta_archivo_folio)

In [0]:
#Carga los datos del segundo dataframe a la tabla CIERREN.TRAFOGRAL_ARCHIVO_FOLIO
db.write_data(df_inserta_archivo_folio,"CIERREN.TRAFOGRAL_ARCHIVO_FOLIO", "default", "append")

In [0]:
#temp_delta = 'D' + '_' + params.sr_folio
#db.write_delta(temp_delta, db.read_data("default", statement_001), "overwrite")
#if conf.debug:
    #display(db.read_delta(temp_delta))

In [0]:
Notify.send_notification("INFO", params) 

In [0]:
CleanUpManager.cleanup_notebook(locals())